In [1]:
import backtrader as bt
import requests
import datetime
import time

class SimpleLivingData0(bt.feed.DataBase):
    params = (
        ('sleeptime', 10),
        ('symbol', 'ixic'),
    )
    
    def __init__(self, symbol, **kwargs):
        '''symbol: ixic hkHSI'''
        super().__init__(**kwargs)
        
        self.symbol = symbol
        
        # used to stop feed when no more new data from web
        self.last_dt = datetime.datetime.now()
        
    def start(self):
        super().start()
        # add filter adjust time
        self.resample(timeframe=self.p.timeframe, compression=self.p.compression)

    def stop(self):
        pass
    
    def islive(self):
        ''' tell cerebro this is a living feed'''
        return True

    def _load(self):
        # request data
        msg = self._req_data()
        if msg is None:
            # No more data
            return False
        
        # fill the lines
        self.lines.datetime[0] = self.date2num(msg['datetime'])
        self.lines.open[0] = msg['open']
        self.lines.high[0] = msg['high']
        self.lines.low[0] = msg['low']
        self.lines.close[0] = msg['close']
        self.lines.volume[0] = msg['volume']
        self.lines.openinterest[0] = -1

        # Say success
        return True

    
    def _req_data(self):
        '''internal function used to request data from the web
            base on the self.symbol it will fetch different data source'''
        
        # sleep some time
        time.sleep(self.p.sleeptime)
        
        msg = dict()
        if self.symbol == 'ixic':
            # ixic
            ret = requests.get('https://hq.sinajs.cn/list=gb_ixic')
            lstr = ret.text.split(',')

            dt = datetime.datetime.strptime(lstr[3],'%Y-%m-%d %H:%M:%S')
            if self.last_dt == dt:
                return None
            else:
                self.last_dt = dt
            msg['datetime'] = dt
            msg['open'] = float(lstr[5])
            msg['high'] = float(lstr[6])
            msg['low'] = float(lstr[7])
            msg['close'] = float(lstr[1])
            msg['volume'] = int(lstr[10])
            
        elif self.symbol == 'hkHSI':
            # hkHSI
            ret = requests.get('https://hq.sinajs.cn/list=rt_hkHSI')
            lstr = ret.text.split(',')
            
            dt = datetime.datetime.strptime(lstr[17]+' '+lstr[18], '%Y/%m/%d %H:%M:%S')
            if self.last_dt == dt:
                return None
            else:
                self.last_dt = dt
            msg['datetime'] = dt
            msg['open'] = float(lstr[2])
            msg['high'] = float(lstr[4])
            msg['low'] = float(lstr[5])
            msg['close'] = float(lstr[6])
            msg['volume'] = int(float(lstr[11]))
        
        return msg
        

In [2]:
class SimpleLivingData(bt.feed.DataBase):
    '''
    爬取新浪接口的实时行情
    '''
    # 定义参数
    params = (
        ('sleeptime', 10),
        ('symbol', 'ixic'),
        ('backfill_start', True),  # do backfilling at the start
        ('backfill_from', None),  # additional data source to do backfill from
    )
    # States for the Finite State Machine in _load
    _ST_FROM, _ST_START, _ST_LIVE, _ST_HISTORBACK, _ST_OVER = range(5)

    def __init__(self, symbol, **kwargs):
        '''symbol: ixic hkHSI'''
        super().__init__(**kwargs)

        self.symbol = symbol

        # used to stop feed when no more new data from web
        # self.last_dt = datetime.datetime.now()
        s = requests.session()
        # 每次数据传输前客户端要和服务器建立TCP连接，为节省传输消耗，默认为keep-alive，
        # 即连接一次，传输多次，然而在多次访问后不能结束并回到连接池中，导致不能产生新的连接
        s.keep_alive = False  # 关闭多余连接

    def start(self):
        super().start()
        if self.p.backfill_from is not None:
            self._state = self._ST_FROM
            self.p.backfill_from.setenvironment(self._env)
            self.p.backfill_from._start()
        else:
            self._state = self._ST_START  # initial state for _load

        # add filter adjust time
        self.resample(timeframe=self.p.timeframe, compression=self.p.compression)
        if self._state == self._ST_START:
            self._start_finish()  # to finish initialization
            self._st_start()

    def _st_start(self):

        if self.p.backfill_start:
            self.put_notification(self.DELAYED)
        # print(self.num2date(self.p.backfill_from.datetime[0]))
        if self.p.backfill_from is not None:
            # self.last_dt = self.num2date(self.p.backfill_from.datetime[0])
            self.last_dt = self.p.backfill_from.datetime.datetime(0)
        else:
            self.last_dt = datetime.datetime.now()
        self._state = self._ST_LIVE
        return True  # no return before - implicit continue

    def stop(self):
        pass

    def islive(self):
        '''tell cerebro this is a living feed'''
        return True

    def _load(self):
        if self._state == self._ST_LIVE:
            # request data
            msg = self._req_data()
            if msg is None:
                # No more data
                # return False
                return
            # fill the lines
            print(msg)
            self.lines.datetime[0] = self.date2num(msg['datetime'])
            self.lines.open[0] = msg['open']
            self.lines.high[0] = msg['high']
            self.lines.low[0] = msg['low']
            self.lines.close[0] = msg['close']
            self.lines.volume[0] = msg['volume']
            self.lines.openinterest[0] = -1

            # Say success
            return True

        elif self._state == self._ST_FROM:
            if not self.p.backfill_from.next():
                # additional data source is consumed
                self._state = self._ST_START

            # copy lines of the same name
            for alias in self.lines.getlinealiases():
                lsrc = getattr(self.p.backfill_from.lines, alias)
                ldst = getattr(self.lines, alias)

                ldst[0] = lsrc[0]

            return True

        elif self._state == self._ST_START:
            if not self._st_start():
                return False

    def _req_data(self):
        '''internal function used to request data from the web
            based on the self.symbol it will fetch different data source'''
        
        # sleep some time
        time.sleep(self.p.sleeptime)

        msg = dict()
        if self.symbol == 'ixic':
            # icix
            url = 'https://hq.sinajs.cn/list=gb_ixic'
            with requests.get(url) as response:
                response.encoding = "UTF-8"
                lstr = response.text.split(',')
            if lstr:
                dt = datetime.datetime.strptime(lstr[3], '%Y-%m-%d %H:%M:%S')
                print(lstr[3])
                print(dt)
                print(self.last_dt)
                if self.last_dt == dt:
                    return None
                else:
                    self.last_dt = dt
                msg['datetime'] = dt
                msg['open'] = float(lstr[5])
                msg['high'] = float(lstr[6])
                msg['low'] = float(lstr[7])
                msg['close'] = float(lstr[1])
                msg['volume'] = int(lstr[10])
        elif self.symbol == 'hkHSI':
            # hkHSI
            url = 'https://hq.sinajs.cn/list=rt_hkHSI'
            with requests.get(url) as response:
                response.encoding = "UTF-8"
                lstr = response.text.split(',')
            if lstr:
                dt = datetime.datetime.strptime(lstr[17] + ' ' + lstr[18], '%Y/%m/%d %H:%M:%S')
                if self.last_dt == dt:
                    return None
                else:
                    self.last_dt = dt
                msg['datetime'] = dt
                msg['open'] = float(lstr[2])
                msg['high'] = float(lstr[4])
                msg['low'] = float(lstr[5])
                msg['close'] = float(lstr[6])
                msg['volume'] = float(lstr[11])

        elif self.symbol == 'btcbitstamp':
            # BTC
            # url = 'https://quotes.sina.cn/fx/api/openapi.php/BtcService.getMinKline?symbol=btcbitstamp&scale=1&datalen=1',
            url = 'http://quotes.sina.cn/fx/api/openapi.php/BtcService.getMinKline?symbol=btcbitstamp&scale=1&datalen=1'
            try:
                # with requests.get(url, verify=False) as ret:
                with requests.get(url) as response:
                    response.encoding = "UTF-8"
                    # print(json.loads(ret.text)['result']['data'])
                    # print(len(json.loads(ret.text)['result']['data']))
                    lstr = json.loads(response.text)['result']['data'][0]
                if lstr:
                    dt = datetime.datetime.strptime(lstr['d'], '%Y-%m-%d %H:%M:%S')
                    # print(lstr['d'])
                    # print(dt)
                    # print(self.last_dt)
                    if self.last_dt == dt:
                        return None
                    else:
                        self.last_dt = dt
                    msg['datetime'] = dt
                    msg['open'] = float(lstr['o'])
                    msg['high'] = float(lstr['h'])
                    msg['low'] = float(lstr['l'])
                    msg['close'] = float(lstr['c'])
                    msg['volume'] = int(lstr['v'])
            except Exception as ex:
                print(ex)
                return None
        elif self.symbol == 'BTC':
            # BTC
            url = 'https://hq.sinajs.cn/list=btc_btcbitstamp'

            with requests.get(url) as response:
                response.encoding = "UTF-8"
                lstr = re.split(',|"', response.text)
            if lstr:
                # print(lstr)
                # print(type(lstr))
                # print(lstr[0])
                dttm = lstr[12] + ' ' + lstr[1]
                dt = datetime.datetime.strptime(dttm, '%Y-%m-%d %H:%M:%S')
                if self.last_dt == dt:
                    return None
                else:
                    self.last_dt = dt
                    msg['datetime'] = dt
                    msg['open'] = float(lstr[6])
                    msg['high'] = float(lstr[7])
                    msg['low'] = float(lstr[8])
                    msg['close'] = float(lstr[9])
                    msg['volume'] = int(lstr[5])
        elif self.symbol == 'AGu':
            # AGu

            url = 'http://money.finance.sina.com.cn/quotes_service/api/json_v2.php/CN_MarketData.getKLineData?symbol=sz300598&scale=5&ma=no&datalen=30'
            with requests.get(url) as response:
                response.encoding = "UTF-8"
                lstr = json.loads(response.text)[29]
            print(lstr)
            if lstr:
                dt = datetime.datetime.strptime(lstr['day'], '%Y-%m-%d %H:%M:%S')
                if self.last_dt == dt:
                    return None
                else:
                    self.last_dt = dt
                msg['datetime'] = dt
                msg['open'] = float(lstr['open'])
                msg['high'] = float(lstr['high'])
                msg['low'] = float(lstr['low'])
                msg['close'] = float(lstr['close'])
                msg['volume'] = int(lstr['volume'])

        return msg

In [3]:
import backtrader as bt
import pyautogui

class AutoGuiBroker(bt.brokers.BackBroker):
    ''' 
    '''
    def __init__(self):
        super().__init__()
        
        anchor_loc = pyautogui.locateCenterOnScreen('./loc_pic/anchor.png')
        assert(anchor_loc is not None)

        # calc button loc positions
        self.buy_but = (anchor_loc[0]-50,anchor_loc[1]+50)
        self.sell_but = (anchor_loc[0]-180,anchor_loc[1]+50)
        self.market_tab = (anchor_loc[0]-210,anchor_loc[1]+100)
        self.num_input = (anchor_loc[0]-50,anchor_loc[1]+175)
    
    def submit(self, order, check=True):
        super().submit(order, check)
        
        # click market anyway
        pyautogui.click(self.market_tab)
        size = str(abs(order.size))
        if 0==order.ordtype:
            # buy order
            pyautogui.click(self.buy_but)
            pyautogui.click(self.num_input)
            pyautogui.press('backspace',presses=5)
            pyautogui.write(size)
            pyautogui.press('enter')
        else:
            # sell order
            pyautogui.click(self.sell_but)
            pyautogui.click(self.num_input)
            pyautogui.press('backspace',presses=5)
            pyautogui.write(size)
            pyautogui.press('enter')

ModuleNotFoundError: No module named 'pyautogui'

In [4]:
import backtrader as bt
import datetime
import pandas as pd

# Create a Stratey
class ToyStrategy(bt.Strategy):
    '''
    Empty strategy is used to study indicators
    '''
    params = (
        ('none', 0),
    )
    
    def log(self, txt):
        ''' Logging function for this strategy'''
        dt = self.datas[0].datetime.datetime(0)
        print('%s, %s' % (dt.isoformat(), txt))
        
    def __init__(self):
        #bt.talib.KAMA(self.data0.close, timeperiod=3)
        pass
        
    def notify_order(self, order):
        #print(order)
        pass
        
    def next(self):
        self.log('next:{}-{}'.format(self.data0.close[0], len(self)))
        l = len(self)
        print('l:',l)
        if 1==l:
            self.log('buy1')
            self.buy(size=1)
        elif 2==l:
            self.log('sell2')
            self.sell(size=2)
        elif 3==l:
            self.log('close')
            self.close()
    
    def stop(self):
        pass

In [6]:
cerebro = bt.Cerebro(oldtrades=False, stdstats=False)

# from simplelivingfeed import SimpleLivingData
# from autoguibroker import AutoGuiBroker

feed = SimpleLivingData('AGu', timeframe=bt.TimeFrame.Seconds, compression=30)
cerebro.adddata(feed, name='simple living')
#cerebro.resampledata(feed, name='cy_weekly', timeframe=bt.TimeFrame.Minutes, compression=1)

cerebro.addstrategy(ToyStrategy)

#cerebro.broker = AutoGuiBroker()

# init cash
cerebro.broker.setcash(10000.0)

print('Starting Running')

result = cerebro.run()

print('Finish Running')

Starting Running
{'day': '2023-06-02 15:00:00', 'open': '51.790', 'high': '51.800', 'low': '51.750', 'close': '51.760', 'volume': '235900'}
{'datetime': datetime.datetime(2023, 6, 2, 15, 0), 'open': 51.79, 'high': 51.8, 'low': 51.75, 'close': 51.76, 'volume': 235900}
2023-06-02T15:00:00, next:51.76-1
2023-06-02T15:00:00, buy1
{'day': '2023-06-02 15:00:00', 'open': '51.790', 'high': '51.800', 'low': '51.750', 'close': '51.760', 'volume': '235900'}
{'day': '2023-06-02 15:00:00', 'open': '51.790', 'high': '51.800', 'low': '51.750', 'close': '51.760', 'volume': '235900'}
{'day': '2023-06-02 15:00:00', 'open': '51.790', 'high': '51.800', 'low': '51.750', 'close': '51.760', 'volume': '235900'}
{'day': '2023-06-02 15:00:00', 'open': '51.790', 'high': '51.800', 'low': '51.750', 'close': '51.760', 'volume': '235900'}


KeyboardInterrupt: 

In [53]:
# -*- coding: utf-8 -*-
# This example uses the implicit API, in the future we will have options unified which will make things easier.
# You can check if the unified methods are ready-to-use (createOrder, fetchOrder etc) by checking: `is_unified = exchange.has['option']`

import asyncio
import os
import sys
from pprint import pprint

# root = os.path.dirname(os.path.dirname(os.path.dirname(os.path.abspath(__file__))))
# sys.path.append(root + '/python')

import ccxt.async_support as ccxt  # noqa: E402


async def main():
    exchange = ccxt.binance(config={
    'apiKey': 'CRdqW2S3T5n6WCGor6mirN1J8scS5TJIeqyMF1h5g8oKc9gb6B01hpeL4FLrOc9K',
    'secret': 'QoRbjRtl4Emr0jzpUfRPGNGbhhMfs1h92Q0LKpXDeWWXkW1v5nkUMH7pZnRWmlnJ',
    'enableRateLimit': True,
    'timeout':100000,
    'requests_trust_env':True,
    'options': {
#             'defaultType': 'spot'
            'adjustForTimeDifference':True,
        }
    }) 
    await exchange.load_markets()
    market_id = 'ETH-221028-1700-C'
    symbol = 'ETH/USDT:USDT-221028-1700-C'
    timeframe = '1m'
    since = 1592317127349
    limit = 10
    try:
        response = await exchange.fetch_ohlcv(symbol, timeframe, since, limit)
        # Implicit API:
        # response = await exchange.eapiPublicGetKlines({
        #     'symbol': market_id,
        #     'interval': timeframe,
        #     # 'startTime': since,  # optional
        #     # 'limit': limit,  # optional
        # })
        pprint(response)
    except Exception as e:
        print('fetch_OHLCV() failed')
        print(e)
    await exchange.close()


# asyncio.run(main())
await main()

ExchangeNotAvailable: binance GET https://api.binance.com/sapi/v1/capital/config/getall?timestamp=1686062328735&recvWindow=10000&signature=576bf399ea71e41dd4976250b5d4e4f9d6680124870c5a41d38f57f7705f7ec1

In [36]:
import ccxt

exchange = ccxt.binance(config={
    'apiKey': 'CRdqW2S3T5n6WCGor6mirN1J8scS5TJIeqyMF1h5g8oKc9gb6B01hpeL4FLrOc9K',
    'secret': 'QoRbjRtl4Emr0jzpUfRPGNGbhhMfs1h92Q0LKpXDeWWXkW1v5nkUMH7pZnRWmlnJ',
    'enableRateLimit': True,
    'timeout':10000,
    'options': {
#             'defaultType': 'spot'
            'adjustForTimeDifference':True,
        }
})  # 使用Binance交易所，你也可以选择其他交易所
exchange.load_markets()
# print(exchange.fetch_balance())


RequestTimeout: binance GET https://api.binance.com/sapi/v1/capital/config/getall?timestamp=1685883572844&recvWindow=10000&signature=02a98be985be3a6fa34eaf419acaeef0be9639657e81a1b174d2054c02ed3488

In [46]:
import ccxt
my_exchange = 'Binance' # 一个加密货币交易所的例子

method_to_call = getattr(ccxt, my_exchange.lower()) # 检索与给定交易所名称匹配的ccxt方法

exchange_obj = method_to_call() # 定义一个交换对象

In [47]:
ticker1 = 'BTC' # 货币对的第一个货币

ticker2 = 'USDT' # 货币对的第二个货币

pair_price_data = exchange_obj.fetch_ticker(ticker1+'/'+ticker2)


RequestTimeout: binance GET https://fapi.binance.com/fapi/v1/exchangeInfo

In [2]:
import ccxt

exchange = ccxt.binance(config={
    'apiKey': 'CRdqW2S3T5n6WCGor6mirN1J8scS5TJIeqyMF1h5g8oKc9gb6B01hpeL4FLrOc9K',
    'secret': 'QoRbjRtl4Emr0jzpUfRPGNGbhhMfs1h92Q0LKpXDeWWXkW1v5nkUMH7pZnRWmlnJ',
    'enableRateLimit': True,
    'timeout':100000,
    'requests_trust_env':True,
    'options': {
#             'defaultType': 'spot'
            'adjustForTimeDifference':True,
        }
})  # 使用Binance交易所，你也可以选择其他交易所
# market = exchange.load_markets()
# print(exchange, market)
ticker1 = 'BTC' # 货币对的第一个货币

ticker2 = 'USDT' # 货币对的第二个货币

pair_price_data = exchange.fetch_ticker(ticker1+'/'+ticker2)
print(pair_price_data)

{'symbol': 'BTC/USDT', 'timestamp': 1687093584578, 'datetime': '2023-06-18T13:06:24.578Z', 'high': 26680.0, 'low': 26311.0, 'bid': 26532.34, 'bidVolume': 5.44877, 'ask': 26532.35, 'askVolume': 3.77543, 'vwap': 26507.74900526, 'open': 26531.26, 'close': 26532.35, 'last': 26532.35, 'previousClose': 26531.27, 'change': 1.09, 'percentage': 0.004, 'average': 26531.805, 'baseVolume': 18540.14747, 'quoteVolume': 491457575.655324, 'info': {'symbol': 'BTCUSDT', 'priceChange': '1.09000000', 'priceChangePercent': '0.004', 'weightedAvgPrice': '26507.74900526', 'prevClosePrice': '26531.27000000', 'lastPrice': '26532.35000000', 'lastQty': '0.00112000', 'bidPrice': '26532.34000000', 'bidQty': '5.44877000', 'askPrice': '26532.35000000', 'askQty': '3.77543000', 'openPrice': '26531.26000000', 'highPrice': '26680.00000000', 'lowPrice': '26311.00000000', 'volume': '18540.14747000', 'quoteVolume': '491457575.65532400', 'openTime': '1687007184578', 'closeTime': '1687093584578', 'firstId': '3146827157', 'las

In [52]:
pair_price_data = exchange.fetch_ticker(ticker1+'/'+ticker2)
print(pair_price_data)

{'symbol': 'BTC/USDT', 'timestamp': 1686051214709, 'datetime': '2023-06-06T11:33:34.709Z', 'high': 26817.61, 'low': 25388.0, 'bid': 25692.08, 'bidVolume': 7.62606, 'ask': 25692.09, 'askVolume': 5.86988, 'vwap': 25906.28062917, 'open': 26770.81, 'close': 25692.09, 'last': 25692.09, 'previousClose': 26770.8, 'change': -1078.72, 'percentage': -4.029, 'average': 26231.45, 'baseVolume': 63617.76634, 'quoteVolume': 1648099707.8047569, 'info': {'symbol': 'BTCUSDT', 'priceChange': '-1078.72000000', 'priceChangePercent': '-4.029', 'weightedAvgPrice': '25906.28062917', 'prevClosePrice': '26770.80000000', 'lastPrice': '25692.09000000', 'lastQty': '0.00042000', 'bidPrice': '25692.08000000', 'bidQty': '7.62606000', 'askPrice': '25692.09000000', 'askQty': '5.86988000', 'openPrice': '26770.81000000', 'highPrice': '26817.61000000', 'lowPrice': '25388.00000000', 'volume': '63617.76634000', 'quoteVolume': '1648099707.80475680', 'openTime': '1685964814709', 'closeTime': '1686051214709', 'firstId': '31338

In [5]:
import pandas as pd
import datetime
symbol = 'BTC/USDT'
timeframe = '1m'
since = 1592317127349
limit = 13
data = exchange.fetch_ohlcv(symbol, timeframe=timeframe,  limit=limit)
data
df = pd.DataFrame(data, dtype=float,columns=['begin_time','open','high','low','close','volume'])
df['begin_time'] = pd.to_datetime(df['begin_time'],utc=True, unit='ms').dt.tz_convert('Asia/Shanghai')

print(datetime.datetime.strptime(str(df['begin_time'][0]).ljust(24, "0"),"%Y-%m-%d %H:%M:%S%z") )
print(str(df['begin_time'][0])[:19])
print(datetime.datetime.strptime(str(df['begin_time'][0])[:19], '%Y-%m-%d %H:%M:%S'))
# df['bb'] = pd.to_datetime(df['begin_time'], format='%m/%d/%Y %I:%M:%S')
df['bb'] = df['begin_time'].dt.tz_localize(None)

df

2023-06-18 20:57:00+08:00
2023-06-18 20:57:00
2023-06-18 20:57:00


,begin_time,open,high,low,close,volume,bb
0,2023-06-18 20:57:00+08:00,26540.55,26540.56,26538.00,26538.01,5.59029,2023-06-18 20:57:00
1,2023-06-18 20:58:00+08:00,26538.01,26538.01,26533.63,26533.64,3.98155,2023-06-18 20:58:00
2,2023-06-18 20:59:00+08:00,26533.64,26534.98,26532.32,26534.97,2.85211,2023-06-18 20:59:00
3,2023-06-18 21:00:00+08:00,26534.97,26538.00,26534.97,26538.00,10.04512,2023-06-18 21:00:00
4,2023-06-18 21:01:00+08:00,26537.99,26540.00,26537.99,26539.99,4.08373,2023-06-18 21:01:00
5,2023-06-18 21:02:00+08:00,26539.99,26540.00,26534.00,26534.00,7.06625,2023-06-18 21:02:00
6,2023-06-18 21:03:00+08:00,26534.01,26535.31,26532.00,26535.31,30.03359,2023-06-18 21:03:00
7,2023-06-18 21:04:00+08:00,26535.31,26535.31,26530.07,26530.07,20.30132,2023-06-18 21:04:00
8,2023-06-18 21:05:00+08:00,26530.07,26530.08,26524.95,26524.95,3.82209,2023-06-18 21:05:00
9,2023-06-18 21:06:00+08:00,26524.95,26532.35,26524.95,26527.03,16.28544,2023-06-18 21:06:00


In [112]:
import time
data1 = exchange.fetch_tickers([symbol,])
timestamp = float(data1['BTC/USDT']['timestamp'])
timestamp
timeTuple = time.localtime(timestap/1000)
begin_time = time.strftime('%Y-%m-%d  %H:%M:%S', timeTuple)
print(begin_time)
data1

2023-06-06  23:16:04


{'BTC/USDT': {'symbol': 'BTC/USDT',
  'timestamp': 1686065206893,
  'datetime': '2023-06-06T15:26:46.893Z',
  'high': 26510.95,
  'low': 25351.02,
  'bid': 25935.15,
  'bidVolume': 3.46929,
  'ask': 25935.16,
  'askVolume': 6.9424,
  'vwap': 25760.1744196,
  'open': 26423.99,
  'close': 25935.16,
  'last': 25935.16,
  'previousClose': 26423.98,
  'change': -488.83,
  'percentage': -1.85,
  'average': 26179.575,
  'baseVolume': 72090.91363,
  'quoteVolume': 1857074509.1773193,
  'info': {'symbol': 'BTCUSDT',
   'priceChange': '-488.83000000',
   'priceChangePercent': '-1.850',
   'weightedAvgPrice': '25760.17441960',
   'prevClosePrice': '26423.98000000',
   'lastPrice': '25935.16000000',
   'lastQty': '0.26420000',
   'bidPrice': '25935.15000000',
   'bidQty': '3.46929000',
   'askPrice': '25935.16000000',
   'askQty': '6.94240000',
   'openPrice': '26423.99000000',
   'highPrice': '26510.95000000',
   'lowPrice': '25351.02000000',
   'volume': '72090.91363000',
   'quoteVolume': '1857

In [113]:
import pytz
ts = timestap/1000
tz = pytz.timezone('Asia/Shanghai')
dt = pytz.datetime.datetime.fromtimestamp(ts, tz)
dt.strftime('%Y-%m-%d %H:%M:%S')

'2023-06-06 23:16:04'

In [4]:
apiKey = 'CRdqW2S3T5n6WCGor6mirN1J8scS5TJIeqyMF1h5g8oKc9gb6B01hpeL4FLrOc9K'
secret = 'QoRbjRtl4Emr0jzpUfRPGNGbhhMfs1h92Q0LKpXDeWWXkW1v5nkUMH7pZnRWmlnJ'

import pandas as pd

def get_data_binance(exchange_binance, symbol, timeframe, limit):
    data = exchange_binance.fetch_ohlcv(symbol, timeframe=timeframe,  limit=limit)
    df = pd.DataFrame(data, dtype=float,columns=['begin_time','open','high','low','close','volume'])
    df['begin_time'] = pd.to_datetime(df['begin_time'],utc=True, unit='ms').dt.tz_convert('Asia/Shanghai')
    df['begin_time'] = df['begin_time'].dt.tz_localize(None)
    print(df)
    return df

In [5]:
import ccxt
exchange_binance = ccxt.binance(config={
            'apiKey': apiKey,
            'secret': secret,
            'enableRateLimit': True,
            'timeout':100000,
            'requests_trust_env':True,
            'options': {
                    'adjustForTimeDifference':True,
                },
            'proxies': {
                    'http': 'http://127.0.0.1:10809',
                    'https': 'http://127.0.0.1:10809',
                },    

        })  # 使用Binance交易所，你也可以选择其他交易所

In [6]:
symbol='BTC/USDT'
timeframe = '1m'
limit = 2
df = get_data_binance(exchange_binance=exchange_binance,symbol=symbol,timeframe=timeframe,limit=limit)
df

           begin_time      open      high       low     close    volume
0 2023-07-16 22:23:00  30326.98  30352.36  30326.98  30352.35  19.76121
1 2023-07-16 22:24:00  30352.36  30352.36  30352.35  30352.35   0.14177


,begin_time,open,high,low,close,volume
0,2023-07-16 22:23:00,30326.98,30352.36,30326.98,30352.35,19.76121
1,2023-07-16 22:24:00,30352.36,30352.36,30352.35,30352.35,0.14177


In [7]:
df = get_data_binance(exchange_binance=exchange_binance,symbol=symbol,timeframe=timeframe,limit=limit)
df

           begin_time      open      high       low     close    volume
0 2023-07-16 22:23:00  30326.98  30352.36  30326.98  30352.35  19.76121
1 2023-07-16 22:24:00  30352.36  30352.36  30352.35  30352.35   0.55756


,begin_time,open,high,low,close,volume
0,2023-07-16 22:23:00,30326.98,30352.36,30326.98,30352.35,19.76121
1,2023-07-16 22:24:00,30352.36,30352.36,30352.35,30352.35,0.55756


In [7]:
df.iloc[0]

begin_time    2023-07-16 21:42:00
open                      30310.0
high                     30310.23
low                      30309.99
close                    30310.22
volume                    3.80829
Name: 0, dtype: object

In [8]:
df['begin_time'][1]

Timestamp('2023-07-16 21:43:00')

In [9]:
import datetime
dt1 = datetime.datetime.strptime(str(df['begin_time'][1])[:19], '%Y-%m-%d %H:%M:%S')
dt2 = dt1 + datetime.timedelta(minutes=1)

dt1,dt2

(datetime.datetime(2023, 7, 16, 22, 24),
 datetime.datetime(2023, 7, 16, 22, 25))